# Tigergraph Tutorial

## Overview
Notebook for experimenting with pyTigerGraph


In [1]:
import os
from dotenv import load_dotenv
import pyTigerGraph as tg

load_dotenv()

tg_host = os.environ.get("TG_HOST")
tg_secret = os.environ.get("TG_SECRET")
tg_graph = os.environ.get("TG_GRAPH")
tg_password = os.environ.get("TG_PASSWORD")

print(tg_graph)

conn = tg.TigerGraphConnection(host=tg_host, graphname=tg_graph, password=tg_password)

social


In [2]:
print(conn.gsql('ls', options=[]))

---- Graph social
Vertex Types:
- VERTEX person(PRIMARY_ID name STRING, name STRING, age INT, gender STRING, state STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
Edge Types:
- UNDIRECTED EDGE friendship(FROM person, TO person, connect_day DATETIME)

Graphs:
- Graph social(person:v, friendship:e)
Jobs:
Queries:






In [5]:
print(conn.gsql('use global drop all', options=[]))

Dropping all, about 1 minute ...
Abort all active loading jobs
Try to abort all loading jobs on graph MaritesGraph, it may take a while ...
[ABORT_SUCCESS] No active Loading Job to abort.
Try to abort all loading jobs on graph social, it may take a while ...
[ABORT_SUCCESS] No active Loading Job to abort.
Resetting GPE...
Successfully reset GPE and GSE
Stopping GPE GSE
Successfully stopped GPE GSE in 0.004 seconds
Clearing graph store...
Successfully cleared graph store
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.162 seconds
Everything is dropped.


In [6]:
print(conn.gsql('''
use global
create vertex person (primary_id name string, name string, age int, gender string, state string)

create undirected edge friendship (from person, to person, connect_day datetime)

create graph social (person, friendship)
''', options=[]))

Successfully created vertex types: [person].
Successfully created edge types: [friendship].
Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 36.042 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.159 seconds
The graph social is created.


In [8]:
%%writefile person.csv
name,gender,age,state
Tom,male,40,ca
Dan,male,34,ny
Jenny,female,25,tx
Kevin,male,28,az
Amily,female,22,ca
Nancy,female,20,ky
Jack,male,26,fl

Writing person.csv


In [9]:
%%writefile friendship.csv
person1,person2,date
Tom,Dan,2017-06-03
Tom,Jenny,2015-01-01
Dan,Jenny,2016-08-03
Jenny,Amily,2015-06-08
Dan,Nancy,2016-01-03
Nancy,Jack,2017-03-02
Dan,Kevin,2015-12-30

Writing friendship.csv


In [9]:
secret = conn.createSecret()
conn.getToken(secret=secret)

('591645uorsq834eiae5ulsj03sjp32nf', 1652351265, '2022-05-12 10:27:45')

In [11]:
import pandas as pd

person = pd.read_csv('./person.csv')
person

,name,gender,age,state
0,Tom,male,40,ca
1,Dan,male,34,ny
2,Jenny,female,25,tx
3,Kevin,male,28,az
4,Amily,female,22,ca
5,Nancy,female,20,ky
6,Jack,male,26,fl


In [12]:
conn.upsertVertexDataFrame(df=person, vertexType='person', v_id='name')

7

In [13]:
friendship = pd.read_csv('./friendship.csv')
friendship

,person1,person2,date
0,Tom,Dan,2017-06-03
1,Tom,Jenny,2015-01-01
2,Dan,Jenny,2016-08-03
3,Jenny,Amily,2015-06-08
4,Dan,Nancy,2016-01-03
5,Nancy,Jack,2017-03-02
6,Dan,Kevin,2015-12-30


In [14]:
conn.upsertEdgeDataFrame(
    df=friendship, 
    sourceVertexType='person',
    edgeType='friendship',
    targetVertexType='person',
    from_id='person1', 
    to_id='person2',
    attributes={'connect_day': 'date'})

7

In [16]:
conn.getVertexDataframe('person', select='age,gender,name', where='age > 22')

,v_id,age,gender,name
0,Jenny,25,female,Jenny
1,Jack,26,male,Jack
2,Kevin,28,male,Kevin
3,Dan,34,male,Dan
4,Tom,40,male,Tom


In [17]:
conn.getVertexDataframeById('person', ['Tom', 'Jenny'])

,v_id,name,age,gender,state
0,Tom,Tom,40,male,ca
1,Jenny,Jenny,25,female,tx


In [22]:
conn.getEdgesDataframe('person', 'Tom', 'friendship')

,from_type,from_id,to_type,to_id,connect_day
0,person,Tom,person,Dan,2017-06-03 00:00:00
1,person,Tom,person,Jenny,2015-01-01 00:00:00
